In [0]:
pip install azure-eventhub

Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc8921be-5ea0-4885-8536-ad6d1ed133b4
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
service_credential = dbutils.secrets.get(scope="databricksscopename",key="databricksservicekey")

spark.conf.set("fs.azure.account.auth.type.adlsx5u224gv3xgd2.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.adlsx5u224gv3xgd2.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.adlsx5u224gv3xgd2.dfs.core.windows.net", "5c962712-e111-48b0-b9af-41f03175e216")
spark.conf.set("fs.azure.account.oauth2.client.secret.adlsx5u224gv3xgd2.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.adlsx5u224gv3xgd2.dfs.core.windows.net", "https://login.microsoftonline.com/72f988bf-86f1-41af-91ab-2d7cd011db47/oauth2/token")

In [0]:
import pyspark.pandas as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
from azure.eventhub import EventHubProducerClient, EventHubConsumerClient, EventData

In [0]:
csv_file_path = 'abfss://data@adlsx5u224gv3xgd2.dfs.core.windows.net/sales_orders.csv'  # Replace with your CSV file path

# Read a csv file using pandas
df = ps.read_csv(csv_file_path)
df = df.head(20)

display(df)
# Convert the CSV file into JSON
json_data = df.to_json(orient='records')

clicked_items,customer_id,customer_name,number_of_line_items,order_datetime,order_number,ordered_products,promo_info
"""[[""""AVpfPEx61cnluZ0-gyT9""""","""""34""""]","[""""AVpfuJ4pilAPnD_xhDyM""""","""""98""""]","[""""AVpe6jFBilAPnD_xQxO2""""","""""60""""]","[""""AVpfIODe1cnluZ0-eg35""""","""""49""""]]"""
"""[[""""AVpfdBS41cnluZ0-lBIj""""","""""88""""]]""",4401099,denevi digital imaging,1,01-08-2019 03:27,317568015,"""{""""id"""":""""AVpfdBS41cnluZ0-lBIj"""""
"""[[""""AVpgIu4Q1cnluZ0-xBK-""""","""""13""""]","[""""AVpfeG5oilAPnD_xcTsG""""","""""27""""]","[""""AVqVGaEBv8e3D1O-ldFu""""","""""64""""]","[""""AVpg-Wj61cnluZ0-8sZe""""","""""87""""]"
"""[[""""AVpfWxx7LJeJML437u-H""""","""""52""""]","[""""AVpfvr461cnluZ0-qgHR""""","""""54""""]","[""""AVpiMIyE1cnluZ0-K0TA""""","""""14""""]","[""""AVz5wc0H-jtxr-f30F6_""""","""""84""""]"
"""[[""""AVpfCQslilAPnD_xThwe""""","""""80""""]","[""""AVpiMIyE1cnluZ0-K0TA""""","""""50""""]]""",2732808,als deli,2,01-08-2019 05:03
"""[[""""AVpfDLA0ilAPnD_xT0sq""""","""""21""""]","[""""AVwjdOdov8e3D1O-nnK9""""","""""85""""]","[""""AVpfMVD-ilAPnD_xW6bu""""","""""75""""]]""",3838589,"MORENO, SALVADOR C"
"""[[""""AVpe9oDr1cnluZ0-a1wt""""","""""34""""]","[""""AVpe7vER1cnluZ0-aJu7""""","""""74""""]","[""""AVpfMVD-ilAPnD_xW6bu""""","""""48""""]]""",7159905,"TURNER ALSTON, DENISE"
"""[[""""AVphzZkJ1cnluZ0-Gkcq""""","""""28""""]","[""""AVpe48Es1cnluZ0-ZHZU""""","""""46""""]","[""""AVpge6k2LJeJML43OhAl""""","""""16""""]","[""""AVpfCW42ilAPnD_xTj0y""""","""""70""""]]"""
"""[[""""AVpfMVD-ilAPnD_xW6bu""""","""""36""""]","[""""AVpjedgc1cnluZ0-W4NI""""","""""64""""]]""",8513182,guardian sprinklers inc,2,01-08-2019 07:33
"""[[""""AVpfPEx61cnluZ0-gyT8""""","""""35""""]","[""""AVpfuJ4pilAPnD_xhDyN""""","""""99""""]","[""""AVpe6jFBilAPnD_xQxO3""""","""""61""""]","[""""AVpfIODe1cnluZ0-eg35""""","""""50""""]]"""


In [0]:
print(json_data)

[{"clicked_items":"\"[[\"\"AVpfPEx61cnluZ0-gyT9\"\"","customer_id":"\"\"34\"\"]","customer_name":"[\"\"AVpfuJ4pilAPnD_xhDyM\"\"","number_of_line_items":"\"\"98\"\"]","order_datetime":"[\"\"AVpe6jFBilAPnD_xQxO2\"\"","order_number":"\"\"60\"\"]","ordered_products":"[\"\"AVpfIODe1cnluZ0-eg35\"\"","promo_info":"\"\"49\"\"]]\""},{"clicked_items":"\"[[\"\"AVpfdBS41cnluZ0-lBIj\"\"","customer_id":"\"\"88\"\"]]\"","customer_name":"4401099","number_of_line_items":"denevi digital imaging","order_datetime":"1","order_number":"01-08-2019 03:27","ordered_products":"317568015","promo_info":"\"{\"\"id\"\":\"\"AVpfdBS41cnluZ0-lBIj\"\""},{"clicked_items":"\"[[\"\"AVpgIu4Q1cnluZ0-xBK-\"\"","customer_id":"\"\"13\"\"]","customer_name":"[\"\"AVpfeG5oilAPnD_xcTsG\"\"","number_of_line_items":"\"\"27\"\"]","order_datetime":"[\"\"AVqVGaEBv8e3D1O-ldFu\"\"","order_number":"\"\"64\"\"]","ordered_products":"[\"\"AVpg-Wj61cnluZ0-8sZe\"\"","promo_info":"\"\"87\"\"]"},{"clicked_items":"\"[[\"\"AVpfWxx7LJeJML437u-H\"\"

In [0]:
event_hub_connection_string = 'Endpoint=sb://streamdata-2f774f-ns.servicebus.windows.net/;SharedAccessKeyName=rule;SharedAccessKey=dS5c+1Wi3xKqKYcz0vD0+mE06dnAp5t2u+AEhFp4vFQ=;EntityPath=test'  # Replace with your connection string
event_hub_name = 'test'



producer = EventHubProducerClient.from_connection_string(event_hub_connection_string, eventhub_name=event_hub_name)

# Publish the JSON as events
try:
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json_data))
    producer.send_batch(event_data_batch)
except Exception as e:
    print('Error occurred: ', str(e))
finally:
    producer.close()